**Лабораторная работа №5** Вариант 9

**Цель лабораторной работы:** научиться производить кластерный анализ данных с
использованием метода К-средних.

Набор данных служит хорошим введением в реализацию алгоритмов машинного обучения, поскольку требует элементарной очистки данных, имеет легко понимаемый список переменных и оптимальный размер между слишком игрушечным и слишком громоздким. Данные содержат информацию по экономике и безработице.

Атрибуты:


*   **Name** - год наблюдения и инициалы астронома.
*   **Node (degrees)** - угол в плоскости орбиты Земли, под которым малая планета пересекает орбиту Земли.
*   **Inclination (Degrees)** - угол между орбитами Земли и малой планеты.
*   **Axis (Astronomic Units)** - максимальное расстояние между малой планетой и Солнцем, деленное на соответствующую величину для Земли.

**1. Импортируем библиотеки и загружаем данные.**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

df = pd.read_csv('/content/planets.csv')
df.head()

**2. Получим информацию о датасете.**

In [ ]:
df.info()

**3. Проверяем наличие пропущенных значений и выбросов.**

In [ ]:
print(df.isnull().sum())

**4. Применяем операцию нормализации для численной устойчивости.**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler

**5. Приведем данные к единому масштабу.**

In [ ]:
X = scaler.fit_transform(df[['Node (degrees)', 'Inclination (Degrees)', 'Axis (Astronomic Units)']])
X[:5]

**6. Определяем оптимальное количество кластеров с помощью метода локтя**

In [ ]:
from sklearn.cluster import KMeans

wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Метод локтя')
plt.xlabel('Количество кластеров')
plt.ylabel('WCSS')
plt.show()

**7. Обучаем модель кластеризации.**

In [ ]:
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, random_state=42)
kmeans.fit(X)

**8. Предсказываем кластеры и визуализируем результаты.**

In [ ]:
labels = kmeans.predict(X)

plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis')
plt.title('Кластеры малых планет')
plt.xlabel('Node (degrees)')
plt.ylabel('Inclination (Degrees)')
plt.show()

**9. Интерпретируем результаты и улучшаем модель при необходимости.**

In [ ]:
X[:5]

In [ ]:
labels

In [ ]:
df['Label'] = labels
df.head()

In [ ]:
numeric_columns = ['Node (degrees)', 'Inclination (Degrees)', 'Axis (Astronomic Units)']
cluster_groups = df[numeric_columns + ['Label']].groupby('Label')
cluster_groups.mean()

In [ ]:
plt.scatter(X[labels == 0, 0], X[labels == 0, 1], s = 100, c = 'red', label = 'Cluster 0')
plt.scatter(X[labels == 1, 0], X[labels == 1, 1], s = 100, c = 'blue', label = 'Cluster 1')
plt.scatter(X[labels == 2, 0], X[labels == 2, 1], s = 100, c = 'green', label = 'Cluster 2')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
           s = 300, c = 'yellow', label = 'Centroids')
plt.title('Кластеры малых планет')
plt.xlabel('Node (degrees)')
plt.ylabel('Inclination (Degrees)')
plt.legend()
plt.show()

**10. Снижение размерности набора данных с помощью метода PCA.**

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('Clusters of Minor Planets')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
numeric_columns = ['Node (degrees)', 'Inclination (Degrees)', 'Axis (Astronomic Units)']
cluster_groups = df[numeric_columns + ['Label']].groupby('Label')
cluster_groups.mean()

In [ ]:
df.head(50)

In [ ]:
# Вывод планет в каждом кластере
for cluster_num in range(5):
  print(f"\nCluster {cluster_num} planets")
  print(df[df['Label'] == cluster_num]['Name'].values)